In [0]:
%sql
-- ===============================================
-- 1) Rango de fechas desde vehiculos
-- ===============================================
CREATE OR REPLACE TEMP VIEW v_rango_fechas AS
SELECT
  MIN(fecha_compra) AS min_fecha,
  MAX(fecha_compra) AS max_fecha
FROM tfm_gold.vehiculos
WHERE fecha_compra IS NOT NULL;

-- ===============================================
-- 2) Calendario completo (min..max) como vista
-- ===============================================
CREATE OR REPLACE TEMP VIEW v_calendario AS
SELECT explode(
         sequence(
           (SELECT min_fecha FROM v_rango_fechas),
           (SELECT max_fecha FROM v_rango_fechas),
           INTERVAL 1 DAY
         )
       ) AS d;

-- ===============================================
-- 3) Inserción incremental:
--    Solo agrega días que NO existan aún en dim_fecha
-- ===============================================
INSERT INTO tfm_gold.dim_fecha
SELECT
  d AS Fecha,
  YEAR(d) AS Anio,
  MONTH(d) AS MesNumero,
  CASE MONTH(d)
    WHEN 1 THEN 'enero' WHEN 2 THEN 'febrero' WHEN 3 THEN 'marzo'
    WHEN 4 THEN 'abril' WHEN 5 THEN 'mayo' WHEN 6 THEN 'junio'
    WHEN 7 THEN 'julio' WHEN 8 THEN 'agosto' WHEN 9 THEN 'septiembre'
    WHEN 10 THEN 'octubre' WHEN 11 THEN 'noviembre' WHEN 12 THEN 'diciembre'
  END AS MesNombre,
  CASE MONTH(d)
    WHEN 1 THEN 'ene' WHEN 2 THEN 'feb' WHEN 3 THEN 'mar'
    WHEN 4 THEN 'abr' WHEN 5 THEN 'may' WHEN 6 THEN 'jun'
    WHEN 7 THEN 'jul' WHEN 8 THEN 'ago' WHEN 9 THEN 'sep'
    WHEN 10 THEN 'oct' WHEN 11 THEN 'nov' WHEN 12 THEN 'dic'
  END AS MesAbrev,
  CAST(date_format(d,'yyyyMM') AS INT) AS AnioMes,
  DAY(d) AS Dia,
  (CASE WHEN dayofweek(d)=1 THEN 7 ELSE dayofweek(d)-1 END) AS DiaSemanaNum,
  CASE (CASE WHEN dayofweek(d)=1 THEN 7 ELSE dayofweek(d)-1 END)
    WHEN 1 THEN 'lunes'
    WHEN 2 THEN 'martes'
    WHEN 3 THEN 'miércoles'
    WHEN 4 THEN 'jueves'
    WHEN 5 THEN 'viernes'
    WHEN 6 THEN 'sábado'
    WHEN 7 THEN 'domingo'
  END AS DiaSemanaNombre,
  CONCAT('Q', QUARTER(d)) AS Trimestre,
  CONCAT('S', CASE WHEN MONTH(d) <= 6 THEN 1 ELSE 2 END) AS Semestre
FROM v_calendario c
LEFT ANTI JOIN tfm_gold.dim_fecha f
  ON f.Fecha = c.d;